In [ ]:
# default_exp data.features

# Featurizing Time Series

> Functions used to transform time series into a dataframe that can be used to create tabular dataloaders.

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.data.core import *
from tsai.data.tabular import *

In this case we are using tsfresh that is one of the most widely known libraries used to create features from time series. You can get more details about this library here: https://tsfresh.readthedocs.io/en/latest/

In [ ]:
#export
import tsfresh
from tsfresh import extract_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters

In [ ]:
#export
@delegates(extract_features.__init__)
def get_ts_features(X:Union[np.ndarray, torch.Tensor], y:Union[None, np.ndarray, torch.Tensor]=None, features:Union[str, dict]='min', 
                    n_jobs:Optional[int]=None, **kwargs):
    """
    Args:
        X: np.array or torch.Tesnor of shape [samples, dimensions, timesteps].
        y: Not required for unlabeled data. Otherwise, you need to pass it.
        features: 'min', 'efficient', 'all', or a dictionary. Be aware that 'efficient' and 'all' may required substantial memory and time.
    """
    df = to_tsfresh_df(X)
    n_jobs = ifnone(n_jobs, defaults.cpus)
    if 'default_fc_parameters' in kwargs.keys(): default_fc_parameters = default_fc_parameters 
    elif features == 'min': default_fc_parameters = MinimalFCParameters()
    elif features == 'efficient': default_fc_parameters = EfficientFCParameters()
    elif features == 'all': default_fc_parameters = ComprehensiveFCParameters()
    else: default_fc_parameters = None
    df = tsfresh.extract_features(df, column_id="id", n_jobs=n_jobs, default_fc_parameters=default_fc_parameters, **kwargs)
    if y is not None:
        if y.ndim == 1: y = y.reshape(-1,1)
        for i in range(y.shape[-1]):
            df['target' if y.shape[-1] == 1 else f'target_{i}'] = y[:, i]
    return df

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape

(360, 24, 51)

There are 3 levels of fatures you can extract: 'min', 'efficient' and 'all'. I'd encourage you to start with min as feature creation may take a long time. 

In addition to this, you can pass a dictionary to build the desired features (see tsfresh documentation in the link above).

In [ ]:
ts_features_df = get_ts_features(X, y)
ts_features_df.shape

Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.43it/s]


(360, 193)

The 'min' set creates a dataframe with 8 features per channel + 1 per target (total 193) for each time series sample (360).

In [ ]:
cont_names = ts_features_df.columns[:-1]
y_names = 'target'
dls = get_tabular_dls(ts_features_df, splits=splits, cont_names=cont_names, y_names=y_names)
dls.show_batch()

,0__sum_values,0__median,0__mean,0__length,0__standard_deviation,0__variance,0__maximum,0__minimum,1__sum_values,1__median,1__mean,1__length,1__standard_deviation,1__variance,1__maximum,1__minimum,2__sum_values,2__median,2__mean,2__length,2__standard_deviation,2__variance,2__maximum,2__minimum,3__sum_values,3__median,3__mean,3__length,3__standard_deviation,3__variance,3__maximum,3__minimum,4__sum_values,4__median,4__mean,4__length,4__standard_deviation,4__variance,4__maximum,4__minimum,5__sum_values,5__median,5__mean,5__length,5__standard_deviation,5__variance,5__maximum,5__minimum,6__sum_values,6__median,6__mean,6__length,6__standard_deviation,6__variance,6__maximum,6__minimum,7__sum_values,7__median,7__mean,7__length,7__standard_deviation,7__variance,7__maximum,7__minimum,8__sum_values,8__median,8__mean,8__length,8__standard_deviation,8__variance,8__maximum,8__minimum,9__sum_values,9__median,9__mean,9__length,9__standard_deviation,9__variance,9__maximum,9__minimum,10__sum_values,10__median,10__mean,10__length,10__standard_deviation,10__variance,10__maximum,10__minimum,11__sum_values,11__median,11__mean,11__length,11__standard_deviation,11__variance,11__maximum,11__minimum,12__sum_values,12__median,12__mean,12__length,12__standard_deviation,12__variance,12__maximum,12__minimum,13__sum_values,13__median,13__mean,13__length,13__standard_deviation,13__variance,13__maximum,13__minimum,14__sum_values,14__median,14__mean,14__length,14__standard_deviation,14__variance,14__maximum,14__minimum,15__sum_values,15__median,15__mean,15__length,15__standard_deviation,15__variance,15__maximum,15__minimum,16__sum_values,16__median,16__mean,16__length,16__standard_deviation,16__variance,16__maximum,16__minimum,17__sum_values,17__median,17__mean,17__length,17__standard_deviation,17__variance,17__maximum,17__minimum,18__sum_values,18__median,18__mean,18__length,18__standard_deviation,18__variance,18__maximum,18__minimum,19__sum_values,19__median,19__mean,19__length,19__standard_deviation,19__variance,19__maximum,19__minimum,20__sum_values,20__median,20__mean,20__length,20__standard_deviation,20__variance,20__maximum,20__minimum,21__sum_values,21__median,21__mean,21__length,21__standard_deviation,21__variance,21__maximum,21__minimum,22__sum_values,22__median,22__mean,22__length,22__standard_deviation,22__variance,22__maximum,22__minimum,23__sum_values,23__median,23__mean,23__length,23__standard_deviation,23__variance,23__maximum,23__minimum,target
0,-31.687590,-0.449933,-0.621325,51.0,1.131348,1.279949,1.736834,-2.283808,-16.302590,-0.006616,-0.319659,51.0,1.013854,1.027900,0.882160,-2.065694,-61.100647,-1.349777,-1.198052,51.0,0.568826,0.323564,-0.016052,-1.992497,51.926018,0.842558,1.018157,51.0,0.951547,0.905441,2.367542,-0.546820,-42.782078,-0.791868,-0.838864,51.0,0.893807,0.798892,0.423345,-2.377712,3.175629,-0.184566,0.062267,51.0,0.989049,0.978218,1.566781,-1.496043,-41.555508,-0.725470,-0.814814,51.0,0.294545,0.086757,-0.329041,-1.266218,-22.076424,-0.679716,-0.432871,51.0,0.444817,0.197862,0.220083,-1.348709,-34.127652,-0.693618,-0.669170,51.0,0.159478,0.025433,-0.327424,-1.065561,42.054775,0.859676,0.824603,51.0,0.210473,0.044299,1.589184,0.574062,-36.303024,-0.757318,-0.711824,51.0,0.358840,0.128766,-0.210639,-1.655061,20.224211,0.330459,0.396553,51.0,0.265177,0.070319,0.765451,-0.176420,-33.866142,-0.602986,-0.664042,51.0,0.847634,0.718483,1.148847,-1.766993,-16.736234,-0.200915,-0.328161,51.0,0.758470,0.575276,0.647467,-1.550873,-53.966419,-1.090789,-1.058165,51.0,0.298422,0.089056,-0.241720,-1.626734,46.019440,0.752719,0.902342,51.0,0.637167,0.405982,1.743538,-0.238214,-39.030647,-0.623139,-0.765307,51.0,0.677121,0.458493,0.133827,-1.924617,6.346338,-0.071636,0.124438,51.0,0.767484,0.589032,1.147075,-1.334729,-34.825542,-0.449683,-0.682854,51.0,0.963484,0.928301,1.143906,-2.074954,-13.005811,-0.033904,-0.255016,51.0,0.915946,0.838957,0.793895,-1.785950,-58.590382,-1.264853,-1.148831,51.0,0.441024,0.194502,0.006054,-1.804485,48.2920

In [ ]:
x_cat, x_cont, yb = first(dls.train)
x_cont

tensor([[ 0.2510, -1.0216,  0.2510,  ..., -0.8254, -1.2280, -0.4989],
        [ 2.8093,  2.8782,  2.8093,  ...,  1.1049,  0.1915, -1.3763],
        [-0.3896, -0.3842, -0.3896,  ..., -0.8492, -1.7897, -0.7025],
        ...,
        [-0.2363, -0.2105, -0.2363,  ..., -0.6975, -0.5374,  0.4494],
        [ 0.8455,  0.3421,  0.8455,  ..., -0.3739, -0.0638, -0.1428],
        [ 0.5340,  0.8307,  0.5340,  ...,  0.4782,  1.3826,  1.5660]])

In [ ]:
from tsai.models.utils import *
from tsai.models.TabModel import *
model = build_tabular_model(TabModel, dls=dls)
learn = Learner(dls, model, metrics=[accuracy, RocAuc()])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.806789,1.778912,0.216667,0.614370,00:00
1,1.781897,1.676545,0.422222,0.909481,00:00
2,1.733134,1.567385,0.533333,0.945778,00:00
3,1.678977,1.493117,0.566667,0.949370,00:00
4,1.632174,1.471912,0.577778,0.950074,00:00


In [ ]:
b = first(dls.train)
model(*b[:-1]).shape

torch.Size([64, 6])

In [ ]:
#hide
out = create_scripts(); beep(out)